In [17]:
from fitESPconstrained import *

In [18]:
from ase.build import molecule
from ase.visualize import view

In [19]:
benzene = molecule('C6H6')

In [20]:
for a in benzene:
    print(a)

Atom('C', [0.0, 1.395248, 0.0], index=0)
Atom('C', [1.2083200000000001, 0.69762400000000002, 0.0], index=1)
Atom('C', [1.2083200000000001, -0.69762400000000002, 0.0], index=2)
Atom('C', [0.0, -1.395248, 0.0], index=3)
Atom('C', [-1.2083200000000001, -0.69762400000000002, 0.0], index=4)
Atom('C', [-1.2083200000000001, 0.69762400000000002, 0.0], index=5)
Atom('H', [0.0, 2.4823599999999999, 0.0], index=6)
Atom('H', [2.1497869999999999, 1.2411799999999999, 0.0], index=7)
Atom('H', [2.1497869999999999, -1.2411799999999999, 0.0], index=8)
Atom('H', [0.0, -2.4823599999999999, 0.0], index=9)
Atom('H', [-2.1497869999999999, -1.2411799999999999, 0.0], index=10)
Atom('H', [-2.1497869999999999, 1.2411799999999999, 0.0], index=11)


In [24]:
test = benzene[0]

In [29]:
test.symbol

'C'

In [22]:
view(benzene,viewer='ngl')

HBox(children=(NGLWidget(), VBox(children=(Dropdown(description='Show', options=('All', 'C', 'H'), value='All'), FloatSlider(value=0.8, description='Ball size', max=1.5, step=0.01)))))

In [11]:
infile_cost_h5 = 'sandbox/system100.cost_ua_neg.h5'

In [12]:
A_horton, B_horton, C_horton, N_horton = \
        read_horton_cost_function(file_name = infile_cost_h5)

In [14]:
X_unconstrained, A_unconstrained, B_unconstrained = \
            unconstrainedMinimize(A_matrix = A_horton,
                        b_vector = B_horton,
                        C_scalar = C_horton,
                        debug    = True)

In [37]:
X_unconstrained

array([-0.32995227,  0.13740187,  0.14611325, -0.08869054,  0.14535466,
        0.24916843,  0.05968685, -0.37675413,  0.15361529,  0.15995601,
        0.01357123,  0.125004  ,  0.51767463, -0.39850823, -0.57901354,
        0.4731755 ,  0.25134963, -0.40564907,  0.24553691,  0.1618362 ,
        0.41776803,  0.37848833, -0.30819086, -0.54716458,  0.42707974,
        0.34040823, -0.56427557,  0.3822506 ,  0.36537504,  0.39866003,
       -0.30676208,  0.19923517, -0.18901963,  0.12772008,  0.34203855,
        0.03372989, -0.51238329,  0.08519556,  0.05474783,  0.4376331 ,
       -0.02318244,  0.43436063, -0.35509081, -0.4637006 ,  0.23398196,
        0.38617388, -0.40725561,  0.31117669,  0.3333818 ,  0.31632931,
        0.52635899, -0.3571091 , -0.5934792 ,  0.38190876,  0.46811357,
       -0.649581  ,  0.39829966,  0.37581468,  0.35040656, -0.03113381,
        0.08108863, -0.17824511,  0.20334884,  0.20046629, -0.19176515,
        0.14120303, -0.01167768,  0.05180708, -0.17387741,  0.20

In [31]:
for i, a in enumerate(benzene):
    a.charge = X_unconstrained[i]

In [42]:
for a in benzene:
    print("{}, {: .3f}".format(a.symbol,a. charge))

C, -0.330
C,  0.137
C,  0.146
C, -0.089
C,  0.145
C,  0.249
H,  0.060
H, -0.377
H,  0.154
H,  0.160
H,  0.014
H,  0.125
